In [1]:
import json
import numpy as np
from seismic import SeismicIndexRaw
from tqdm.notebook import tqdm
import pandas as pd

In [2]:
doc_path = "/data1/knn_datasets/sparse_datasets/beir/beir_spladev3_fromCarlos_CROSSCHECK-MRR-BEFORE-USE/beir/quora/data/documents.bin"

index = SeismicIndexRaw.build(doc_path, n_postings=3500)


Building the index...
Configuration { pruning: GlobalThreshold { n_postings: 3500, max_fraction: 1.5 }, blocking: RandomKmeans { centroid_fraction: 0.1, min_cluster_size: 2, clustering_algorithm: RandomKmeansInvertedIndexApprox { doc_cut: 15 } }, summarization: EnergyPreserving { summary_energy: 0.4 }, knn: KnnConfiguration { nknn: 0, knn_path: None }, batched_indexing: None }
Distributing and pruning postings: 0 secs
Number of posting lists: 26595
Building summaries: 2 secs


In [3]:
query_path = "/data1/knn_datasets/sparse_datasets/beir/beir_spladev3_fromCarlos_CROSSCHECK-MRR-BEFORE-USE/beir/quora/data/queries.bin"
groundtruth_path = "/data1/knn_datasets/sparse_datasets/beir/beir_spladev3_fromCarlos_CROSSCHECK-MRR-BEFORE-USE/beir/quora/data/groundtruth.tsv"

In [4]:
def compute_accuracy(gt_pd, res_pd):
    gt_pd_groups = gt_pd.groupby('query_id')['doc_id'].apply(set)
    res_pd_groups = res_pd.groupby('query_id')['doc_id'].apply(set)

    # Compute the intersection size for each query_id in both dataframes
    intersections_size = {
        query_id: len(gt_pd_groups[query_id] & res_pd_groups[query_id]) if query_id in res_pd_groups else 0
        for query_id in gt_pd_groups.index
    }

    # Computes total number of results in the groundtruth
    total_results = len(gt_pd)
    total_intersections = sum(intersections_size.values())
    return total_intersections/total_results

In [5]:
import struct
def read_sparse_vectors_from_binary_file(filename):
    term_id = []
    with open(filename, "rb") as f:
        # Read the number of term vectors (sparse vectors)
        num_vectors = int.from_bytes(f.read(4), byteorder='little', signed=False)
        for _ in tqdm(range(num_vectors)):
            # Read the length of the integer sequence
            seq_len = int.from_bytes(f.read(4), byteorder='little', signed=False)
            
            # Read the integer sequence (indices)
            int_seq = np.array([int.from_bytes(f.read(4), byteorder='little', signed=False) for _ in range(seq_len)], dtype=np.int32)
            
            # Read the float sequence (values)
            float_seq = np.array([struct.unpack('f', f.read(4))[0] for _ in range(seq_len)], dtype=np.float32)
            
            # Append the parsed term (as two lists: indices and values) to term_id
            term_id.append((int_seq, float_seq))
    
    return term_id

In [6]:
queries = read_sparse_vectors_from_binary_file(query_path)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [7]:
docs = read_sparse_vectors_from_binary_file(doc_path)

  0%|          | 0/522931 [00:00<?, ?it/s]

In [8]:
groundtruth = pd.read_csv(groundtruth_path, sep="\t", names=["query_id", "doc_id", "rank", "score"])


In [10]:
query = queries[0]
query

(array([ 3268,  6451,  6454,  6781,  6782,  6783,  6947,  7529,  7543,
         7560, 10470, 11032, 15182, 15230, 16160, 17719, 18538, 19268,
        19272, 20386, 20390, 20409, 20411, 22405, 24536, 26268, 26272],
       dtype=int32),
 array([1.5485512 , 0.8366989 , 0.29458022, 1.3666275 , 1.1879934 ,
        0.5701008 , 0.2724157 , 0.6082087 , 0.20773558, 0.64504516,
        0.6225774 , 0.19207929, 0.25022486, 0.05737934, 0.20898588,
        0.0873253 , 0.5944445 , 0.06783415, 0.32910192, 1.8867625 ,
        1.8483115 , 2.277586  , 0.73086494, 1.3352437 , 0.40709767,
        0.7613972 , 0.49380526], dtype=float32))

In [16]:
for component, value in sorted(zip(*query), key=lambda x:x[1], reverse=True):
    component_id = component.item()
    current_posting = index.get_doc_ids_in_postings(component_id)
    break

In [19]:
def sparse_dot_product(vec1, vec2):

    components1, values1 = vec1
    components2, values2 = vec2
    

    common_components = np.intersect1d(components1, components2)
    
    if len(common_components) == 0:
        return 0.0
    

    idx1 = np.searchsorted(components1, common_components)
    idx2 = np.searchsorted(components2, common_components)
    

    return np.sum(values1[idx1] * values2[idx2])

In [20]:
def sparse_dot_product_dict(vec1, vec2):
    """
    Alternative implementation using dictionary for potentially better performance
    with very sparse vectors or unsorted component arrays.
    """
    components1, values1 = vec1
    components2, values2 = vec2
    
    # Create dictionary for first vector
    vec1_dict = dict(zip(components1, values1))
    
    # Compute dot product by iterating through second vector
    dot_product = 0.0
    for comp, val in zip(components2, values2):
        if comp in vec1_dict:
            dot_product += vec1_dict[comp] * val
    
    return dot_product

In [69]:
from collections import defaultdict

distances = []

n_docs = len(current_posting)
doc_ids = list(current_posting)

n_neighbors = 10

neighbors = defaultdict(list)

for i in tqdm(range(n_docs), desc="Computing distances"):
    current_distances = []
    for j in range(i + 1, n_docs):  # j starts from i+1 to avoid duplicates
        doc_id1 = doc_ids[i]
        doc_id2 = doc_ids[j]
        distance = sparse_dot_product_dict(docs[doc_id1], docs[doc_id2])
        current_distances.append(distance.item())
    neighbors_pos = np.argsort(current_distances)[::-1][:n_neighbors]
    neighbors_doc_ids = [current_posting[idx] for idx in neighbors_pos]
    neighbors[current_posting[i]] = neighbors_doc_ids
    

Computing distances:   0%|          | 0/5250 [00:00<?, ?it/s]

In [75]:
docs[current_posting[0]]

(array([ 3268,  5783,  5790,  5870,  5955,  7382,  8269,  9422,  9651,
         9655, 11577, 12370, 13133, 15098, 15585, 16425, 16426, 17301,
        18215, 18547, 18585, 18848, 19006, 20409, 20898, 21077, 21461,
        22275, 22281, 24478, 25235, 25693, 25809, 25812, 25939, 26268,
        26272], dtype=int32),
 array([1.3567029 , 1.2116898 , 0.4431409 , 0.40483412, 0.11302987,
        1.3565516 , 0.21892956, 0.25243187, 1.0681663 , 0.07211731,
        0.20025276, 2.6297421 , 0.24361883, 0.01153325, 0.5079931 ,
        1.7307117 , 0.7348002 , 0.17940268, 0.4347429 , 0.1419206 ,
        0.00742344, 0.11844329, 0.19764292, 2.1712842 , 0.03636141,
        0.4001993 , 0.02758142, 0.10035896, 0.27606964, 0.45732707,
        0.24872115, 0.49463516, 0.39716342, 0.05756392, 0.63710576,
        0.55043817, 0.67720306], dtype=float32))

In [81]:
docs[current_posting[0]]

(array([ 3268,  5783,  5790,  5870,  5955,  7382,  8269,  9422,  9651,
         9655, 11577, 12370, 13133, 15098, 15585, 16425, 16426, 17301,
        18215, 18547, 18585, 18848, 19006, 20409, 20898, 21077, 21461,
        22275, 22281, 24478, 25235, 25693, 25809, 25812, 25939, 26268,
        26272], dtype=int32),
 array([1.3567029 , 1.2116898 , 0.4431409 , 0.40483412, 0.11302987,
        1.3565516 , 0.21892956, 0.25243187, 1.0681663 , 0.07211731,
        0.20025276, 2.6297421 , 0.24361883, 0.01153325, 0.5079931 ,
        1.7307117 , 0.7348002 , 0.17940268, 0.4347429 , 0.1419206 ,
        0.00742344, 0.11844329, 0.19764292, 2.1712842 , 0.03636141,
        0.4001993 , 0.02758142, 0.10035896, 0.27606964, 0.45732707,
        0.24872115, 0.49463516, 0.39716342, 0.05756392, 0.63710576,
        0.55043817, 0.67720306], dtype=float32))

In [86]:
sorted_docs = sorted(current_posting, key= lambda x: docs[x][1] [docs[x][0]==component_id], reverse=True)[:10]

In [129]:

n_choices = 20
starting_points = np.random.choice(current_posting, n_choices)
#starting_points = sorted_docs

results = []

counter = 0
stop_at = 50

visited = set()

for starting_point in starting_points:

    current_node = starting_point
    counter = 0
    while(counter < stop_at):
        current_neighbors = neighbors[current_node]
        local_results = []
        for neigh in current_neighbors:
            if not neigh in visited:
                local_results.append((neigh, sparse_dot_product_dict(query, docs[neigh])))
                visited.add(neigh)
        if len(local_results) == 0:
            current_node = np.random.choice(current_posting, 1).item()
            continue
        results.extend(local_results)
        index_to_jump = np.argmax([x[1] for x in local_results])
        current_node = current_neighbors[index_to_jump]

        counter += len(current_neighbors)

In [130]:
sorted(results, key=lambda x: x[1], reverse=True)[:10]

[(381484, np.float32(21.436327)),
 (322658, np.float32(20.468542)),
 (39017, np.float32(20.261549)),
 (12006, np.float32(20.156263)),
 (220500, np.float32(20.12595)),
 (46938, np.float32(20.061787)),
 (370899, np.float32(19.81274)),
 (112732, np.float32(19.738323)),
 (45866, np.float32(19.65855)),
 (338747, np.float32(19.521))]

In [126]:
len(visited)

703

In [95]:
exh_results = []
for doc_id in current_posting:
    exh_results.append((doc_id, sparse_dot_product_dict(query, docs[doc_id])))

In [96]:
sorted(exh_results, key=lambda x: x[1], reverse=True)[:10]

[(162780, np.float32(23.165792)),
 (235727, np.float32(23.12265)),
 (168781, np.float32(22.835157)),
 (61789, np.float32(22.481089)),
 (315859, np.float32(22.479797)),
 (323649, np.float32(22.194365)),
 (156921, np.float32(22.173252)),
 (274984, np.float32(22.1589)),
 (260910, np.float32(22.11858)),
 (323648, np.float32(21.927666))]